In [1]:
from elasticsearch import Elasticsearch
import os

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [3]:
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [4]:
bill_filenames = []
for file in os.listdir("../ustawy"):
    if file.endswith(".txt"):
        bill_filenames.append(os.path.join("../ustawy", file))

In [ ]:
es.indices.create(
    index='bills',
    body={
      'settings': {
        'analysis': {
            'analyzer': {
                'default': {
                    'type': 'custom',
                    'tokenizer': 'standard',
                    'filter': ['synonym_filter', 'lowercase', 'morfologik_stem']
                }
            },
            'filter': {
                'synonym_filter': {
                    'type': 'synonym',
                    'synonyms':[
                        'kpk, kodeks postępowania karnego',
                        'kpc, kodeks postępowania cywilnego',
                        'kk, kodeks karny',
                        'kc, kodeks cywilny'
                    ]
                }
            }
        }
      },
      'mappings':{
          'properties': {
              'filename': {'type': 'keyword'},
              'content': {'type': 'text'}
          }
      }
  }
)

In [ ]:
for bill_name in bill_filenames:
    with open(bill_name) as b_file:
        bill = {
            'filename': bill_name[10:],
            'content': b_file.read()
        }
        es.index(index='bills', body=bill)

### Determine the number of legislative acts containing the word ustawa (in any form).


In [6]:
ustawa_res = es.search(index="bills", body={'query': {'match': {'content':'ustawa'}}})
ustawa_res['hits']['total']

{'value': 1179, 'relation': 'eq'}

### Determine the number of legislative acts containing the words kodeks postępowania cywilnego in the specified order, but in an any inflection form.

In [7]:
kpc_res = es.search(index="bills", body={'query': {'match_phrase': {'content':'kodeks postępowania cywilnego'}}})
kpc_res['hits']['total']

{'value': 100, 'relation': 'eq'}

### Determine the number of legislative acts containing the words wchodzi w życie (in any form) allowing for up to 2 additional words in the searched phrase.

In [8]:
wwz_q = {
    'query': {
        'match_phrase':{
            'content': {
                'query': 'wchodzi w życie',
                'slop': 2
            }
        }
    }
}
wwz_res = es.search(index="bills", body=wwz_q)
wwz_res['hits']['total']

{'value': 1175, 'relation': 'eq'}

### Determine the 10 documents that are the most relevant for the phrase konstytucja.

In [9]:
konst_res = es.search(index="bills", body={"query": {"match": {'content':'konstytucja'}}})
for res in konst_res['hits']['hits'][:10]:
    print(res['_source']['filename'],' : ', res['_score'])

1997_629.txt  :  6.8715763
2000_443.txt  :  6.667747
1997_604.txt  :  6.637779
1996_350.txt  :  6.631979
1997_642.txt  :  6.256356
2001_23.txt  :  6.0651193
1996_199.txt  :  5.9354324
1999_688.txt  :  5.8562775
2001_1082.txt  :  5.4733624
1997_681.txt  :  5.4733624


### Print the excerpts containing the word konstytucja (up to three excerpts per document) from the previous task.

In [10]:
exc_q = {
    'query': {
        'match': {
            'content': {
                'query': 'konstytucja'
            }
        }
    },
    'size': 10,
    '_source': [ 'filename' ],
    'highlight': {
        'number_of_fragments': 3,
        'fragment_size': 100,
        'fields': {
            'content': {} 
        }
    }
}

exc_res = es.search(index="bills", body=exc_q)
for res in exc_res['hits']['hits']:
    print(res['_source']['filename'],' : ', res['_score'])
    for highlight in res['highlight']['content']:
        print(highlight)
    print('\n')

1997_629.txt  :  6.8715763
o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej
W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>
Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie


2000_443.txt  :  6.667747
umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>
międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>


1997_604.txt  :  6.637779
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działa

## Podsumowanie
Możemy zobaczyć że przykład znaleziony jako pierwszy zawiera słowo konstytucja w róznych formach aż pięć razy, dziesiąty już tylko dwa.